# 10. ACP

Dans ce notebook, nous allons appliquer une ACP sur différents datasets en utilisant la librairie [scikit-learn](https://scikit-learn.org/stable/).

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale, normalize, StandardScaler

## 10.1 ACP pour la compréhension et l'analyse d'un dataset

Nous allons appliquer une ACP sur un jeu de données décrivant quelques statistiques sur des pays. L'objectif est d'identifier de trouver une catégorisation des pays en fonction de ces donnés.

Charger le dataset contenu dans le fichier `data/country-data.csv` dans une dataframe pandas, visualisez un échantillon de ces données, afficher quelques statistiques (mean, std...) et donnez sa taille.

Nous nous intéressons à présent aux corrélations linéaires entre les variables de ce dataset à l'aide de la méthode [`pandas.DataFrame.corr`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html).

Afficher la matrice des corrélations avec la méthode [`sns.heatmap`](https://seaborn.pydata.org/generated/seaborn.heatmap.html). Vous pouvez utiliser le code `plt.figure(figsize=(10,10))` au préalable pour obtenir une figure plus lisible. Pour afficher cette matrice, les paramètres intéressants de `sns.heatmap` sont `square`, `annot` et `cmap` (utilisez l'espace de couleur `sns.diverging_palette(20, 220, n=200)` pour plus de lisibilité).

Afficher les individus du dataset pour quelques paires de variables fortement corrélées. Vous pouvez utiliser un [scatter plot plotly](https://plotly.com/python/line-and-scatter/). 

Quelles conclusions pouvezèvous tirer des variables et des individus ?

Créez une variable `X` contenant les données numériques des individus (i.e. : sans la colonne `country`). Puis appliquez la méthode `fit_transform` de la classe [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) de scikit-lean, stockez le résultat dans une nouvelle variable `X_std`.

Afficher la distribution des variables de ce dataset avant et après la standardisation en utilisant [`pandas.DataFrame.plot.density`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.density.html) (les paramètres `sharex=True,figsize=(12,5)` rendront ce graphique plus lisible).

Sur le graphique de la distribution des variables avant standardisation, certaines variables dont la plage des valeurs est trop différentes des autres rendrait ce graphique illisible. Pensez à afficher un graphique différent pour chaque groupe de variables ayant une plage de valeurs proche.

Appliquez la méthode `fit` de la classe [sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) pour calculer les composantes principales. Puis afficher les valeurs des attributs `components_`, `explained_variance_` et `explained_variance_ratio_`. A quoi correspondent ces attributs ?

La fonction suivante vous permet d'afficher un graphique représentant la variance expliquée par les différentes composantes principales et la variance cumulée. Utilisez cette fonction sur votre ACP. Combien faut-il conserver de cmposantes principales pour expliquer plus de 80% de la variabilité des données ?

In [ ]:
def display_scree_plot(pca):
    '''Display a scree plot for the pca'''

    scree = pca.explained_variance_ratio_*100
    plt.figure(figsize=(15,8))
    plt.bar(np.arange(len(scree))+1, scree)
    plt.plot(np.arange(len(scree))+1, scree.cumsum(),c="red",marker='o')
    plt.xlabel("Number of principal components")
    plt.ylabel("Percentage explained variance")
    plt.title("Scree plot")
    plt.show(block=False)

La fonction suivante vous permet d'afficher le cercle des corrélations des variables. Ses paramètres principaux sont :
* `pcs` : ndarray, les composantes principales
* `n_comp` : int, le nombre de composantes
* `pca` : sklearn.decomposition.PCA, l'ACP
* `axis_ranks` : list, les indices des paires d'axes à afficher (chaque paire affichera un nouveau cercle de corrélation), exemple : [(0,1)]
* `labels` : list, le nom des variables

Appliquez la pour visualiser la projection des variables dans le premier et le second plan factoriel. Interprétez les composantes principales affichées.

In [ ]:
def display_circles(pcs, n_comp, pca, axis_ranks, labels=None, label_rotation=0, lims=None):
    """Display correlation circles, one for each factorial plane"""

    # For each factorial plane
    for d1, d2 in axis_ranks: 
        if d2 < n_comp:

            # Initialise the matplotlib figure
            fig, ax = plt.subplots(figsize=(10,10))

            # Determine the limits of the chart
            if lims is not None :
                xmin, xmax, ymin, ymax = lims
            elif pcs.shape[1] < 30 :
                xmin, xmax, ymin, ymax = -1, 1, -1, 1
            else :
                xmin, xmax, ymin, ymax = min(pcs[d1,:]), max(pcs[d1,:]), min(pcs[d2,:]), max(pcs[d2,:])

            # Add arrows
            # If there are more than 30 arrows, we do not display the triangle at the end
            if pcs.shape[1] < 30 :
                plt.quiver(np.zeros(pcs.shape[1]), np.zeros(pcs.shape[1]),
                   pcs[d1,:], pcs[d2,:], 
                   angles='xy', scale_units='xy', scale=1, color="grey")
                # (see the doc : https://matplotlib.org/api/_as_gen/matplotlib.pyplot.quiver.html)
            else:
                lines = [[[0,0],[x,y]] for x,y in pcs[[d1,d2]].T]
                ax.add_collection(LineCollection(lines, axes=ax, alpha=.1, color='black'))
            
            # Display variable names
            if labels is not None:  
                for i,(x, y) in enumerate(pcs[[d1,d2]].T):
                    if x >= xmin and x <= xmax and y >= ymin and y <= ymax :
                        plt.text(x, y, labels[i], fontsize='14', ha='center', va='center', rotation=label_rotation, color="blue", alpha=0.5)
            
            # Display circle
            circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
            plt.gca().add_artist(circle)

            # Define the limits of the chart
            plt.xlim(xmin, xmax)
            plt.ylim(ymin, ymax)
        
            # Display grid lines
            plt.plot([-1, 1], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-1, 1], color='grey', ls='--')

            # Label the axes, with the percentage of variance explained
            plt.xlabel('PC{} ({}%)'.format(d1+1, round(100*pca.explained_variance_ratio_[d1],1)))
            plt.ylabel('PC{} ({}%)'.format(d2+1, round(100*pca.explained_variance_ratio_[d2],1)))

            plt.title("Correlation Circle (PC{} and PC{})".format(d1+1, d2+1))
            plt.show(block=False)

Appliquez la fonction `transform` sur votre objet PCA pour appliquer l'ACP sur le dataset centré réduit et stockez le résultats dans une variable `X_pca`. Puis crééz une dataframe contenant le résultat de l'ACP.

Afficher la projection des individus sur les deux premiers plans factoriels. Vous pouvez utiliser un [scatter plot plotly](https://plotly.com/python/line-and-scatter/) avec la dataframe du résultat de l'ACP (pour faciliter la lecture du graphique, vous pouvez utiliser les paramètres `hover_name` ou `text` en utilisant la colonne `country` de votre dataframe d'origine).

Liez ce que vous observez avec l'analyse du cercle des corrélation de ce plan factoriel. Vous pouvez faire de même pour d'autres plans factoriels.

## 10.2 ACP sur des données synthétiques

Nous allons illustrer l'effet de la normalisation et de la mise à l'échelle sur l'ACP appliqué à un dataset synthétique.

Pour cela on se dote d'un dataset contenant 4 variables respectant une distribution normale et une 5ème valant 0 ou 3.

La cellule suivante créé un tel dataset en utilisant numpy:

In [ ]:
np.random.seed(123)    # For reproducibility

N = 200
P = 5
rho = 0.5

X = np.random.normal(size=[N,P])
X = np.append(X, 3*np.random.choice(2, size = [N,1]), axis = 1)
X

Examiner votre dataset et ses propriétés en crééant une dataframe à partir de `X`.

Afficher la distribution des variables de ce dataset en utilisant [`pandas.DataFrame.plot.density`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.density.html) (les paramètres `sharex=True,figsize=(12,5)` rendront ce graphique plus lisible).

Appliquer une ACP sans aucun pré-traitement. Que constatez-vous ?

Appliquez une ACP après avoir réduit vos données ([`sklearn.preprocessing.normalize`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html)). Que constatez-vous ?

Appliquez une ACP après avoir centré vos données ([`sklearn.preprocessing.scale`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html)). Que constatez-vous ?

## 10.3 Appliquez une ACP sur des variations de cours d'actions

Appliquez une ACP sur le dataset contenu dans le fichier `data/company-stock-movements-2010-2015-incl.csv` qui contient les variations quotidiennes des valeurs de titres boursiers en fin de séance par rapport à la valeur de la veille de 2010 à 2015. Puis affichez les variations de titres de bourse proches dans votre nouvel espace.

Note : précisez à Pandas lors de la lecture du fichier CSV que la première colonne contient l'index (paramètre `index_col=0`).

Pour visualiser ces variations pour un ou plusieurs titres vous pouvez utiliser le code suivant (une fois votre dataframe chargée) : `df.loc[['Goldman Sachs', 'Amazon']].transpose().plot(figsize=(15, 7))`.

Attention : l'analyse des cercles de corrélations n'est pas utile pour cette exercice.

**Bonus : Clustering hierarchique**

En utilisant [`scipy.cluster.hierarchy.dendrogram`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) et [`scipy.cluster.hierarchy.linkage`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) réalisez un clustering hiérarchique et visualiez le dendrogramme. Puis utilisez [`sklearn.cluster.AgglomerativeClustering`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) pour affecter un cluster à chaque titre boursier. Enfin commentez les résultats obtenus.